Analyse exploiratoire de données 

In [920]:
import pandas as pd
import numpy as np


Importation données Personnel


In [921]:
data= pd.read_csv('C:/Users/HP/Desktop/mon_stage_senegal/data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20012002.csv' , encoding='latin1' ,sep=';')

In [922]:
print(data.columns) 

Index(['NUMERO', 'NUMERO_TABLE', 'INE', 'NUMERO_IDENTITIFICATION', 'NOM',
       'PRENOM', 'DATE_DE_NAISSANCE', 'LIEU_DE_NAISSANCE',
       'MAIL_INSTITUTIONNEL', 'SEXE', 'MENTION_BACC', 'ANNEE_BACC',
       'PAYS_DE_NAISSANCE', 'NATIONALITE', 'SERIE_BACC', 'REGION_DE_NAISSANCE',
       'ETABLISSMENT_CODE', 'ETABLISSEMENT', 'NIVEAU_SECTION',
       'SIGLE_NIVEAU_SECTION', 'COHORTE', 'ETAT_INSCRIPTION',
       'ANNEE_INSCRIPTION', 'ANNEE_UNIVERSITAIRE', 'TYPE_FORMATION',
       'CODE_NIVEAU', 'NIVEAU', 'DATE_INSCRIPTION', 'NIVEAU LMD',
       'NIVEAU LMD ET NON LMD', 'SYSTEME', 'DEPARTEMENT FORMATION'],
      dtype='object')


In [923]:
data.info()
data1 = data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29094 entries, 0 to 29093
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NUMERO                   29094 non-null  object 
 1   NUMERO_TABLE             0 non-null      float64
 2   INE                      1913 non-null   object 
 3   NUMERO_IDENTITIFICATION  29094 non-null  object 
 4   NOM                      29094 non-null  object 
 5   PRENOM                   29094 non-null  object 
 6   DATE_DE_NAISSANCE        29094 non-null  object 
 7   LIEU_DE_NAISSANCE        28990 non-null  object 
 8   MAIL_INSTITUTIONNEL      3444 non-null   object 
 9   SEXE                     29094 non-null  object 
 10  MENTION_BACC             28760 non-null  object 
 11  ANNEE_BACC               28815 non-null  float64
 12  PAYS_DE_NAISSANCE        29094 non-null  object 
 13  NATIONALITE              29094 non-null  object 
 14  SERIE_BACC            

ici on constate que il y'a 29094 lignes et 32 colonnes 

In [ ]:
# Création de fonction pour gérer les valeurs manquantes
def supprimer_valeur_manquante(data):
    #supprimer la colonne numero_identitification   
    data.drop(columns=['NUMERO_IDENTITIFICATION'], inplace=True)   
    data.columns = data.columns.str.strip().str.replace(' ', '_')  # Remplacer les espaces dans les noms de colonnes par des underscores (_)
    data.columns = data.columns.str.lower()  # Convertir les noms de colonnes en minuscules
    data.dropna(thresh=data.shape[0] * 0.15, axis=1, inplace=True)  # # Supprimer les colonnes qui ont plus de 85% de valeurs manquantes
    return data

# Appliquer la fonction sur le DataFrame data1
data1 = supprimer_valeur_manquante(data1)

In [925]:
#verification 
data1.columns  
data1.isnull().sum()/ data1.shape[0] * 100  

numero                    0.000000
nom                       0.000000
prenom                    0.000000
date_de_naissance         0.000000
lieu_de_naissance         0.357462
sexe                      0.000000
mention_bacc              1.148003
annee_bacc                0.958961
pays_de_naissance         0.000000
nationalite               0.000000
serie_bacc                1.495154
region_de_naissance      36.206778
etablissment_code         0.000000
etablissement             0.000000
niveau_section            0.000000
sigle_niveau_section      0.000000
etat_inscription          0.000000
annee_inscription         0.000000
annee_universitaire       0.000000
type_formation            0.000000
code_niveau               0.000000
niveau                    0.000000
date_inscription          0.000000
niveau_lmd               33.886712
niveau_lmd_et_non_lmd     0.000000
systeme                   0.000000
departement_formation     0.000000
dtype: float64

####                                          traitement des valeurs manquantes 

- Remplacer les 0.3% de la colonne lieu de naissance  de valeurs manquantes par la mention "Inconnu".


- Remplacer les valeurs erronées de la colonne mention bac  exemple ta , ho par "Inconnu".
- Les cellules vides seront remplies par ST pour indiquer un passage au Second Tour.


- Remplacer les valeurs manquantes de la colonne annee_bac  par la valeur modale 


- Remplacer les valeurs manquantes de la colone seri _bac  ou incohérentes par Inconnu.

- Supprimer la colonne NIVEAU_LMD car elle est redondante avec la colonne NIVEAU_LMD_ET_NON_LMD, qui contient les mêmes informations

In [926]:
 

def  traiter_valeur_manquante_anormales(data1) : 
     data1['numero'] = data1['numero'].str.replace('.' , '')
     
     data1['lieu_de_naissance'] = data1['lieu_de_naissance'].fillna('Inconnu')  #remplacer par inconnu 
     data1['mention_bacc'] = data1['mention_bacc'].fillna('ST')# les valeurs null par second Tour
     data1['mention_bacc'] = data1['mention_bacc'].replace(['P' , 'Passable'] , 'PA') #les valeurs P
     data1['mention_bacc'] = data1['mention_bacc'].replace(['TH' , 'TR'], 'TB')  #par TR pour tres bien
     # Remplacer tous les autres que AB, BI, TB, PA, ST par "Inconnu"
     mention_valides= ['AB', 'BI', 'TB', 'PA', 'ST']
     data1['mention_bacc'] = data1['mention_bacc'].apply(lambda x: x if x in mention_valides else 'Inconnu')
     data1['annee_bacc'] = pd.to_numeric(data1['annee_bacc'], errors='coerce')
     #corriger les années < 100 en ajoutant 1900
     data1['annee_bacc'] = data1['annee_bacc'].apply(lambda x: x + 1900 if pd.notnull(x) and x < 100 else x)
     #suprimer les valeurs hors(1960 et 2025)
     data1['annee_bacc'] = data1['annee_bacc'].where(data1['annee_bacc'].between(1960, 2025), np.nan)
     #transformer en int
     data1['annee_bacc'] = data1['annee_bacc'].astype('Int64')
     #remplacer les valeurs manquantes par par la valeur la plus fréquente
     data1['annee_bacc'] = data1['annee_bacc'].fillna(data1['annee_bacc'].mode()[0])
     serie_bac_valides =  ['C', 'S2', 'D', 'B', 'G2', 'A3', 'A', "L'1", 'L2', 'NR', 'S1', 'G',
       'A4', 'E', 'A2', 'S3', 'S', 'T1', 'T2', 'a3', 'L1', 'F7', 'F2']
     data1['serie_bacc'] = data1['serie_bacc'].apply(lambda x: x if x in serie_bac_valides else 'Inconnu')
     #supprimer ces valeurs manquantes aussi par Inconnu 
     data1 ['serie_bacc'] =  data1['serie_bacc'] = data1['serie_bacc'].fillna('Inconnu')
     data1['region_de_naissance'] = data1['region_de_naissance'].fillna('Inconnu')  #remplacer par inconnu
     #supprimer une colonne 
     data1.drop(columns=['niveau_lmd'], inplace=True) 
     
     #remplacer les ... dans la colonne departement_formation par Inconnu 
     data1['departement_formation'] = data1['departement_formation'].replace(['...' , '....' , '.....' , 'Inconnu'])
     return data1 
     
     
data1 = traiter_valeur_manquante_anormales(data1)     
     
     
     

C:\Users\HP\AppData\Local\Temp\ipykernel_12992\1857421406.py:30: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  data1['departement_formation'] = data1['departement_formation'].replace(['...' , '....' , '.....' , 'Inconnu'])


In [927]:
#verification 
data1.isnull().sum()/ data.shape[0] * 100
 

numero                   0.0
nom                      0.0
prenom                   0.0
date_de_naissance        0.0
lieu_de_naissance        0.0
sexe                     0.0
mention_bacc             0.0
annee_bacc               0.0
pays_de_naissance        0.0
nationalite              0.0
serie_bacc               0.0
region_de_naissance      0.0
etablissment_code        0.0
etablissement            0.0
niveau_section           0.0
sigle_niveau_section     0.0
etat_inscription         0.0
annee_inscription        0.0
annee_universitaire      0.0
type_formation           0.0
code_niveau              0.0
niveau                   0.0
date_inscription         0.0
niveau_lmd_et_non_lmd    0.0
systeme                  0.0
departement_formation    0.0
dtype: float64

In [928]:
#verifivation 
data1['annee_bacc'].unique()


<IntegerArray>
[1987, 1998, 1997, 1995, 1999, 1991, 1992, 1990, 1989, 1996, 1994, 1993, 1984,
 1988, 1986, 1985, 1978, 2000, 2001, 1976, 1981, 1982, 1979, 1977, 1983, 1975,
 1974, 1972, 1980, 1973, 1964, 1971, 1967, 1970, 1968, 2003, 2002, 1965, 1969,
 2004, 1966, 2011, 2007, 1960]
Length: 44, dtype: Int64

In [929]:
data1['serie_bacc'].unique()

array(['C', 'S2', 'D', 'B', 'G2', 'A3', 'A', "L'1", 'L2', 'Inconnu', 'S1',
       'G', 'A4', 'E', 'A2', 'S3', 'S', 'T1', 'T2', 'a3', 'F7', 'F2'],
      dtype=object)

In [930]:
#verification 
data1['mention_bacc'].unique()

array(['PA', 'AB', 'BI', 'ST', 'TB', 'Inconnu'], dtype=object)

Supprimer tous les accents  et les caractères spéciaux  present dans les donnees textuelles afin d'assurer une uniformisation des valeurs

In [931]:
#supprimer les accents et les espaces 
import unicodedata
def supprimer_accents_colonnes_textes(data):
    #supprimer les accents dans les colonnes de type texte 
    colonnes_textes = data.select_dtypes(include=['object']).columns 
    for c in colonnes_textes:
        data[c] = data[c].apply(
            lambda text: ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c)) 
            if isinstance(text, str) else text
        )
    return data

# Appliquer la fonction sur le DataFrame data1
data1 = supprimer_accents_colonnes_textes(data1)

In [932]:
#verification
data1['nationalite'].unique()

array(['Senegalaise', 'Camerounaise', 'Beninoise', 'Mauritanienne',
       'Marocaine', 'Tchadienne', 'Gabonaise', 'Burkinabe',
       'Congolaise(Brazzaville)', 'Congolaise(RDC)', 'Djiboutienne',
       'Nigerienne', 'Malienne', 'Ivoirienne', 'Togolaise',
       'Guineenne (conakry)', 'Centrafricaine', 'Americaine',
       'Tanzanienne', 'Burundaise', 'Tunisienne', 'Comorienne',
       'Francaise', 'Rwandaise', 'Iranienne', 'Algerienne', 'Gambienne',
       'Soudanaise', 'Palestinienne', 'Turque', 'Ghaneenne', 'Malgache',
       'Namibienne', 'Bissau-guineenne', 'Cap-verdienne', 'Colombienne',
       'Egyptienne', 'Mauricienne', 'Afghane', 'Belge'], dtype=object)

- suprimer les doublons 

In [933]:
data1['numero'] = data1['numero'].astype(str).str.strip() 
data1[data1.duplicated(subset=['numero'], keep=False)]

#supprimer les doublons


,numero,nom,prenom,date_de_naissance,lieu_de_naissance,sexe,mention_bacc,annee_bacc,pays_de_naissance,nationalite,...,etat_inscription,annee_inscription,annee_universitaire,type_formation,code_niveau,niveau,date_inscription,niveau_lmd_et_non_lmd,systeme,departement_formation
14,1995002X1,DIANKHA,Babacar,02/01/1969,KAOLACK,M,PA,1992,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,23/01/2002,M1,LMD,Langues Anciennes
15,1995002X1,DIANKHA,Babacar,02/01/1969,KAOLACK,M,PA,1992,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,3,Troisieme Annee,20/03/2002,3A,Classique,Langues Anciennes
58,199700B68,SOW,Ibrahima,18/01/1974,DAKAR,M,PA,1994,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,9,Neuvieme annee,22/11/2001,9A,Classique,Medecine interne et Specialites medicales
61,199700C79,NDONGO,Souhaibou,22/06/1975,THILMAKHA,M,AB,1995,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,8,Huitieme annee,14/12/2001,8A,Classique,Medecine interne et Specialites medicales
62,199700BJO,SECK,Lala Bouna,10/01/1970,DAKAR,F,AB,1999,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,8,Huitieme annee,20/11/2001,8A,Classique,Medecine interne et Specialites medicales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28744,199700X0T,NDOUR,Abdoulaye,20/10/1978,DJIDA MOURIDE,M,PA,1996,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,16/03/2002,4A,Classique,Philosophie
28751,199700XGM,SARR,Ngor,06/06/1977,SAOKOM 1,M,PA,1996,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,15/03/2002,4A,Classique,Philosophie
29041,199700ATR,DIENG,Aminata,24/02/1975,DAKAR,F,PA,1992,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,C,7,Septieme Annee,21/12/2001,D,LMD,Medecine
29051,199900S13,ATLASSI,Said,24/07/1966,BENI MELLAL,M,PA,1986,Maroc,Marocaine,...,Inscrit(e),2002,2001-2002,C,7,Septieme Annee,22/03/2002,D,LMD,Medecine


In [934]:
def supprimer_doublons(data):
    # Supprimer les doublons en se basant seulement la première occurrence
    data = data.drop_duplicates(subset=['numero'], keep='first')
    return data

# Appliquer la fonction sur le DataFrame data1
data1 = supprimer_doublons(data1)

In [935]:
#verification
data1[data1.duplicated(subset=['numero'], keep=False)]

,numero,nom,prenom,date_de_naissance,lieu_de_naissance,sexe,mention_bacc,annee_bacc,pays_de_naissance,nationalite,...,etat_inscription,annee_inscription,annee_universitaire,type_formation,code_niveau,niveau,date_inscription,niveau_lmd_et_non_lmd,systeme,departement_formation


- changer le format des dates 

In [936]:
def changer_format_date(data):
    data1['date_de_naissance'] = pd.to_datetime(data1['date_de_naissance'], format='%d/%m/%Y', errors='coerce')
    data1['date_inscription'] = pd.to_datetime(data1['date_inscription'], format='%d/%m/%Y', errors='coerce')
    data1['annee_inscription'] = pd.to_numeric(data1['annee_inscription'], errors='coerce')
    data1['annee_universitaire'] = pd.to_numeric(data1['annee_universitaire'], errors='coerce')
    return data 

#appliquer la fonction sur le DataFrame data1
data1 = changer_format_date(data1)

- Mettre en place un pipeline de nettoyage automatise permettant d’appliquer les mêmes règles de traitement (gestion des valeurs manquantes, correction des formats, suppression des accents et caractères spéciaux ....) à d’autres fichiers similaires 


In [937]:
#pipeline global 
def pipeline_global(df):
    df = supprimer_valeur_manquante(df)
    df = traiter_valeur_manquante_anormales(df)
    df = supprimer_accents_colonnes_textes(df)
    df = supprimer_doublons(df)
    df = changer_format_date(df)
    return df

#fonction pour importer le netoyage 

import os

def importer_netoyage(chemin_fichier, chemin_sortie=None):
    # Déterminer le type de fichier et lire les données
    if chemin_fichier.endswith('.csv'):
        df = pd.read_csv(chemin_fichier , encoding='latin1' , sep=";")
    elif chemin_fichier.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(chemin_fichier)
    else:
        raise ValueError("Format de fichier non pris en charge.")
    
    # Appliquer le pipeline de nettoyage
    df = pipeline_global(df)
    
    # Générer un chemin de sortie par défaut si non fourni
    if chemin_sortie is None:
        nom_fichier = os.path.basename(chemin_fichier)  # Récupérer le nom du fichier d'entrée
        chemin_sortie = f"../data_nettoyer/Etudiants_2001_2024/{os.path.splitext(nom_fichier)[0]}_nettoye.csv"
    
    # Enregistrer le fichier nettoyé
    df.to_csv(chemin_sortie  , index = False , sep = ";" )
    return df


    


In [938]:
#verificarion 
importer_netoyage('../data/Etudiants_2001_2024/Liste_globale_des_etudiants_tous_les_champs20012002.csv' )

C:\Users\HP\AppData\Local\Temp\ipykernel_12992\1857421406.py:30: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  data1['departement_formation'] = data1['departement_formation'].replace(['...' , '....' , '.....' , 'Inconnu'])


,numero,nom,prenom,date_de_naissance,lieu_de_naissance,sexe,mention_bacc,annee_bacc,pays_de_naissance,nationalite,...,etat_inscription,annee_inscription,annee_universitaire,type_formation,code_niveau,niveau,date_inscription,niveau_lmd_et_non_lmd,systeme,departement_formation
0,199900UB6,BA,Alassane,03/05/1967,TENEFOUL,M,PA,1987,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,6,Sixieme Annee,11/12/2001,6A,Classique,Mathematiques-Informatique
1,199800OAE,KANDE,Mountaga,29/10/1977,KOLDA,M,PA,1998,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,12/12/2001,4A,Classique,Genie Civil
2,199700H5O,DIOP,Amadou Lamine,29/12/1974,MEKHE,M,PA,1997,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,10/11/2001,4A,Classique,Economie
3,199700I36,DIOP,Awa,27/03/1971,DAKAR,F,PA,1995,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,5,Cinquieme Annee,01/02/2002,M2,LMD,Institut de Formation en Administration et Cre...
4,199800NC9,DRAME,Mamadou Lamine,09/03/1979,DAKAR,M,PA,1998,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,4,Quatrieme Annee,19/11/2001,4A,Classique,Economie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29088,199800YNQ,NDIAYE,Malick,29/01/1978,OUADIOUR,M,PA,1998,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,2,Deuxieme Annee,19/11/2001,L2,LMD,Mathematiques-Informatique
29089,2000014AO,SOPPO LOBE,Charlotte Vanessa,09/04/1982,DOUALA,F,PA,2000,Cameroun,Camerounaise,...,Inscrit(e),2002,2001-2002,I,2,Deuxieme Annee,29/11/2001,2A,Classique,Pharmacie
29090,199800NC4,FALL,Ndeye Fatoumata,20/10/1978,DAKAR,F,PA,1998,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,3,Troisieme Annee,06/11/2001,3A,Classique,Gestion
29092,200001559,FAYE,Daouda,24/05/1979,POUT-DIACK,M,PA,2000,Senegal,Senegalaise,...,Inscrit(e),2002,2001-2002,I,2,Deuxieme Annee,22/11/2001,L2,LMD,Langue Russe
